# Common logic

In [187]:
# DEFINES
project_name = 'lazyut'


In [188]:
%run Modules/some_helpers.ipynb

project_cppcheck.name = 'CppCheck'
project_cppcheck.branch = 'origin/master'
project_cppcheck.directory = find_existent('C:/Projects/cppcheck', 
                                           'D:/Study/TestCoveredProjects/cppcheck')
project_cppcheck.compiler = find_existent('C:/Qt/Tools/mingw530_32/bin/g++.exe', 
                                          'D:/Soft/Qt/Tools/mingw530_32/bin/g++.exe')
project_cppcheck.src_rel_dir = 'lib'
project_cppcheck.test_rel_dir = 'test'
project_cppcheck.out_dir = project_cppcheck.directory


class ProjectInfo:
    def __init__(self):
        self.list_info = []
        self.n_commits = 10
        self.test_mode = TestMode()
        self.compiler_c = GCC_C
        self.compiler_cxx = GCC_CXX
        self.cmake_generator = 'Unix Makefiles'
        
    def check_attributes(self):
        attributes = ['name', 'branch', 'directory', 'compiler_c', 'compiler_cxx',
                      'src_rel_dir', 'test_rel_dir', 'out_dir', 
                      'builds_dir', 'prepare_fun', 'configure_fun',
                      'build_fun', 'test_fun']
        for a in attributes:
            if not hasattr(self, a):
                raise RuntimeWarning('{}: no attr {}'.format(self.name, a))
    
    def new_iteration(self):
        self.iteration_info = IterationInfo()
        self.list_info.append(self.iteration_info) # add object by reference
    def lazyut_dir(self):
        return os.path.join(self.out_dir, project_name)
    def new_files_dir(self):
        return os.path.join(self.lazyut_dir(), 'files')
    def last_files_dir(self):
        return os.path.join(self.lazyut_dir(), 'last_version', 'files')
    def src_dir(self):
        return os.path.join(self.directory, self.src_rel_dir)
    def test_dir(self):
        return os.path.join(self.directory, self.test_rel_dir)
    def build_dir(self):
        return self.builds_dir + '/' + self.test_mode.directory()
#         return os.path.join(self.builds_dir, self.test_mode.directory())
    def prepare(self):
        self.prepare_fun(self)
    def configure(self):
        self.configure_fun(self)
    def build(self):
        self.build_fun(self)
    def test(self):
        self.test_fun(self)

class IterationInfo:
    __st_idle = 0
    __st_finished_failed = 1
    __st_finished_success = 2
    # global info
    total_ts_retest_all = 0
    total_count_retest_all = 0
    total_failed_all = 0
    total_ts_retest_lazyut = 0
    total_count_retest_lazyut = 0
    total_failed_lazyut = 0
    @staticmethod
    def addFailed(mode):
        if mode == TestMode.retest_all:
            IterationInfo.total_failed_all += 1
        elif mode == TestMode.test_affected:
            IterationInfo.total_failed_lazyut += 1
        else:
            print('addFailed::unknown')
    @staticmethod
    def avg_retest_all():
        if IterationInfo.total_count_retest_all:
            return IterationInfo.total_ts_retest_all / IterationInfo.total_count_retest_all
        return -1
    @staticmethod
    def avg_lazyut():
        if IterationInfo.total_count_retest_lazyut:
            return IterationInfo.total_ts_retest_lazyut / IterationInfo.total_count_retest_lazyut
        return -1
    
    # instance info
    def __init__(self):
        self.state = IterationInfo.__st_idle
    def setCommit(self, commit):
        self.commit = commit
    def setDurationAll(self, ts):
        self.retest_all_ts = ts
        IterationInfo.total_ts_retest_all += ts
        IterationInfo.total_count_retest_all += 1
    def setDurationLazyut(self, ts):
        self.retest_lazyut_ts = ts
        IterationInfo.total_ts_retest_lazyut += ts
        IterationInfo.total_count_retest_lazyut += 1
    def setDurationLazyutAnalysis(self, ts):
        self.lazyut_analysis = ts
    def setDuration(self, ts, mode):
        if mode == TestMode.retest_all:
            self.setDurationAll(ts)
        elif mode == TestMode.test_affected:
            self.setDurationLazyut(ts)
        else:
            print('setDuration::unknown')
    def success(self):
        self.state = IterationInfo.__st_finished_success
    def fail(self):
        self.state = IterationInfo.__st_finished_failed
    
class TestMode:
    unknown = 0
    retest_all = 1
    test_affected = 2
    
    @staticmethod
    def common_directory():
        return 'default'
    
    def __init__(self):
        self.value = TestMode.unknown
    def name(self):
        if self.value == TestMode.unknown:
            return ''
        if self.value == TestMode.retest_all:
            return 're-test all'
        if self.value == TestMode.test_affected:
            return 'lazy testings'
    def directory(self):
        if self.value == TestMode.unknown:
            return ''
        if self.value == TestMode.retest_all:
            return 'default'
        if self.value == TestMode.test_affected:
            return project_name
    def isLazy(self):
        return self.value == TestMode.test_affected
            
def clear_out_file(project):
    with open(project.name + '_output.txt', 'w'): 
        pass
    
def append_out_file(project, line):
    with open(project.name + '_output.txt', 'a+') as f: 
        f.write(line)
        f.write('\n')
        
def is_zero_file(fpath):  
    return os.path.isfile(fpath) and os.path.getsize(fpath) == 0

def removeFilesLazyUT(project):
    shutil.rmtree(project.lazyut_dir(), ignore_errors=True)

def moveFilesLazyUT(project):
    # get list of files
    files = os.listdir(project.new_files_dir())
    # create last_version/files/ directory
    os.makedirs(project.last_files_dir(), exist_ok=True)
    # move files to last_version/files/ directory and overwrite if exists
    for f in files:
        shutil.move(os.path.join(project.new_files_dir(), f), os.path.join(project.last_files_dir(), f))
    
def run_lazyut(project):
    analysis_sdt = timeit.default_timer()
    execute(lazyut + ' -s {} -t {} -o {} -i {} --ignore="cfg/,cfg\,synthetic/,synthetic\"'.format(
                project.src_dir(), project.test_dir(), 
                project.new_files_dir(), project.last_files_dir())
            , shell = True)
    analysis_edt = timeit.default_timer()
    project.iteration_info.setDurationLazyutAnalysis(analysis_edt - analysis_sdt)

def build_project(project):
    # create build directory
    os.makedirs(project.build_dir(), exist_ok=True)
    # setup project
    try:
        project.prepare()
        if project.test_mode.isLazy():
            affected_tests_txt = os.path.join(project.new_files_dir(), 'tests_affected.txt')
            if is_zero_file(affected_tests_txt):
                raise RuntimeWarning('is_zero_file(affected_tests_txt)')
    except:
        project.iteration_info.fail()
        message = 'Project initialization failed! ' + project.iteration_info.commit
        print(message)
        return
    # configure (run cmake, etc.)
    sdt = timeit.default_timer()
    try:
        project.configure()
    except:
        project.iteration_info.fail()
        message = 'configuration failed! ' + project.iteration_info.commit
        print(message)
        return
    configure_edt = timeit.default_timer()
    project.iteration_info.configure_ts = configure_edt - sdt
    try:
        project.build()
    except:
        project.iteration_info.fail()
        message = 'Build failed! ' + project.iteration_info.commit
        print(message)
        return
    build_edt = timeit.default_timer()
    project.iteration_info.build_ts = build_edt - configure_edt

def test_project(project):
    test_sdt = timeit.default_timer()
    try:
        project.test()
    except:
        project.iteration_info.fail()
        message = 'Tests failed! ' + project.iteration_info.commit
        print(message)
        return
    test_edt = timeit.default_timer()
    test_duration = test_edt - test_sdt
    project.iteration_info.test_ts = test_edt - sdt
    output = '{}: configure [{:.5}] build [{:.5}] testing [{:.5}]'.format(
        project.test_mode.name(), project.iteration_info.configure_ts,
        project.iteration_info.build_ts, project.iteration_info.test_ts)
    print(output)
    append_out_file(project, output)
    project.iteration_info.setDuration(test_duration, test_mode)


def run_iteration(project):
    # clear local changes for project
    execute('git reset --hard', cwd = project.directory, shell = True)
    # build default, then build LazyUT in the same build directory
    project.test_mode.value = TestMode.retest_all
    build_project(project)
    project.test_mode.value = TestMode.test_affected
    build_project(project)
    
def run_on_commit(project, commit = None):
    # run re-test all and re-test affected (LazyUT) several times
    # and store evaluation times
    for i in range(0, 3):
        print('iter #%d:' % i)
        project.new_iteration()
        project.iteration_info.setCommit(commit)
        run_iteration(project)
    # print average testing times
    output = 're-test all {:.5}[sec]'.format(IterationInfo.avg_retest_all())
    if avg_retest_lazyut == 0:
        output += ' re-test affected --SKIPPED'
    else:
        output += ' re-test affected (LazyUT) {:.5}[sec]'.format(IterationInfo.avg_retest_lazyut())
    print(output)
    
def git_checkout(project):
    execute('git reset --hard', cwd = project.directory)
    execute('git checkout {}'.format(project.current_commit), cwd = project.directory)
    
# -- SCRIPT --
def apply_lazyut(project):
    # clears output.txt
    clear_out_file(project)
    # clears lazyut/ directory
    removeFilesLazyUT(project)
    # get the list of commits
    rev_list_str = check_output('git rev-list  -n {} {}'.format(
                        project.n_commits, project.branch), cwd = project.directory, shell=True)
    rev_list = rev_list_str.splitlines()[::-1]
    print('{}: total commits: {}'.format(project.name, len(rev_list)))
    # iterate over commits, starting with 'the first commit'
    for commit_bytes in rev_list:
        commit = commit_bytes.decode('utf-8')
        project.current_commit = commit
        print('{}: evaluate commit {}'.format(project.name, project.current_commit))
        # checkout to commit
        git_checkout(project)
        # evaluate multiple iteration on commit
        run_on_commit(project)
        # move LazyUT files from lazyut/files/ to lazyut/last_version/files/
        moveFilesLazyUT(project)
    print('{}: FINISHED!'.format(project.name))

### CMAKE

In [189]:
def CMAKE_GLOB_SRCS(line, tmpf):
    # Add only affected test files
    if 'file(GLOB srcs' in line:
        tmpf.write('    file(STRINGS lazyut/files/tests_affected.txt srcs)\n')
        return
    if 'file(GLOB hdrs' in line:
        tmpf.write('    set(hdrs "") # emptry list\n')
        return
    # Just copy line
    tmpf.write(line)

def prepareCmakeProject(project, method):
    cmakelists_test = os.path.join(project.test_dir(), 'CMakeLists.txt')
    # We are going to modify file, so will use temporary file
    tmpf = tempfile.NamedTemporaryFile(mode='r+')
    # Open for read
    with open(cmakelists_test, 'r') as cmakelists_txt:
        # Firstly check if it is not prepared yet
        # And return if it is
        for line in cmakelists_txt:
            if project_name in line:
                return
        cmakelists_txt.seek(0) # Rewind cmakelists file to beginning
        # Create temporary file read/write
        # Copy input file to temporary file, modifying as we go
        for line in cmakelists_txt:
            method(line, tmpf)
    tmpf.seek(0) # Rewind temporary file to beginning
    # Reopen for write
    with open(cmakelists_test, 'w') as cmakelists_txt:
        for line in tmpf:
            cmakelists_txt.write(line)
    run_lazyut(project)
    
def configure_cmake(project):
    execute('cmake -G "{}" -DCMAKE_CXX_COMPILER={} -DCMAKE_C_COMPILER={} -DBUILD_TESTS=ON {}'.format(
            project.cmake_generator, project.compiler_c, 
            project.compiler_cxx, project.directory), cwd = project.build_dir(), shell = True)
def build_cmake(project):
    execute('cmake --build', timeout=None, cwd = project.build_dir(), shell = True)

### CppCheck

In [190]:
def prepareCppCheck(project):
    cmakelists_test = os.path.join(project.test_dir(), 'CMakeLists.txt')
    # We are going to modify file, so will use temporary file
    tmpf = tempfile.NamedTemporaryFile(mode='r+')
    # Open for read
    with open(cmakelists_test, 'r') as cmakelists_txt:
        # Firstly check if it is not prepared yet
        # And return if it is
        for line in cmakelists_txt:
            if 'shlwapi' in line:
                tmpf.close()
                return 
        cmakelists_txt.seek(0) # Rewind cmakelists file to beginning
        # Create temporary file read/write
        # Copy input file to temporary file, modifying as we go
        for line in cmakelists_txt:
            tmpf.write(line)
            # Link missed libraries (MINGW)
            if 'add_executable(testrunner' in line:
                tmpf.write('    if (MINGW)\n')
                tmpf.write('        target_link_libraries(testrunner shlwapi)\n')
                tmpf.write('    endif()\n')
    tmpf.seek(0) # Rewind temporary file to beginning
    # Reopen for write
    with open(cmakelists_test, 'w') as cmakelists_txt:
        for line in tmpf:
            cmakelists_txt.write(line)
    tmpf.close()         
    


In [191]:
lazyut_cmd = find_existent('C:/Projects/Study/build-LazyRT-5_9-Default/bin/lazyut.exe', 
                           'D:/Study/build-LazyUT-5_7-Debug/bin/lazyut.exe')
GCC_C =  find_existent('C:/Qt/Tools/mingw530_32/bin/gcc.exe', 
                        'D:/Soft/Qt/Tools/mingw530_32/bin/gcc.exe')
GCC_CXX = find_existent('C:/Qt/Tools/mingw530_32/bin/g++.exe', 
                        'D:/Soft/Qt/Tools/mingw530_32/bin/g++.exe')

In [192]:
def prepare_cppcheck(project):
    if project.test_mode.isLazy():
        prepareCmakeProject(project, CMAKE_GLOB_SRCS)
    prepareCppCheck(project)

def configure_cppcheck(project):
    configure_cmake(project)
    
def build_cppcheck(project):
    build_cmake(project)
    
def test_cppcheck(project):
    testrunner = 'testrunner.exe'
    testrunner_dir = os.path.join(project.build_dir(), 'bin')
    execute(testrunner, cwd = testrunner_dir, shell=True)
    
project_cppcheck = ProjectInfo()
project_cppcheck.name = 'CppCheck'
project_cppcheck.branch = 'origin/master'
project_cppcheck.directory = find_existent('C:/Projects/cppcheck', 
                                           'D:/Study/TestCoveredProjects/cppcheck')

project_cppcheck.compiler_c = GCC_C
project_cppcheck.compiler_cxx = GCC_CXX
                            
project_cppcheck.src_rel_dir = 'lib'
project_cppcheck.test_rel_dir = 'test'
project_cppcheck.out_dir = project_cppcheck.directory

project_cppcheck.builds_dir = project_cppcheck.directory + '/builds'

project_cppcheck.prepare_fun = prepare_cppcheck
project_cppcheck.configure_fun = configure_cppcheck
project_cppcheck.build_fun = build_cppcheck
project_cppcheck.test_fun = test_cppcheck


project_cppcheck.check_attributes()
apply_lazyut(project_cppcheck)

print('script finished %s' % str(datetime.datetime.now()) )

CppCheck: total commits: 10
CppCheck: evaluate commit e9a44f70b225c0d4bc240571eb6e4e45466f05ef
iter #0:


TypeError: Can't convert 'NoneType' object to str implicitly

In [ ]:
for x in list_iteration_info:
    print('commit %s' % x.commit)
    print('retest_all_ts %s' % x.retest_all_ts)
    print('retest_lazyut_ts %s' % x.retest_lazyut_ts)
    print('lazyut_analysis %s' % x.lazyut_analysis)
    print()
    